# 畳み込みニューラルネットワーク (CNN)

## 今日の目標
カラー画像の分類に挑戦してみよう

## 目次

- Section1 畳み込みニューラルネットワークとは
  - 1.1 CNNの構成
  - 1.2 畳み込み層 (Convolution Layer)
  - 1.3 プーリング層 (Pooling Layer)
  - 1.4 全結合層 (Dense Layer)
- Section2 Pytorchでの実装
- Section3 汎化性能を上げる

## Section1 CNNとは

### 1.1 CNNの構成

**Convolutional Neural Network (CNN)** は画像認識、音声認識、自然言語処理など、幅広くタスクにおいて使用されているニューラルネットワークの一種です。中でも、画像認識で高い精度を上げたモデルにはCNNをベースに設計されたものが多くあります。

CNNは **畳み込み層 (Convolution layer)** ・ **プーリング層 (Pooling layer)** 呼ばれる層を組み合わせて構築していきます。(2種類の層で構築可能ですが、出力に近い層は **全結合層 (Dense layer)** を組み合わせることがほとんどです。)


<img src="figures/cnn.png" width="60%">

出典：https://deepage.net/deep_learning/2016/11/07/convolutional_neural_network.html

### 1.2 畳み込み層 (Convolution Layer)


#### 1.2.1 畳み込みの考え方

畳み込みは入力のある領域とフィルターの内積を計算することによってフィルターとの類似度のようなものを計算することです。フィルターの数を増やすと多くの特徴を、フィルターのサイズを大きくすると広い範囲の特徴を、小さくすると狭い範囲の特徴を獲得できます。

畳み込みによって得られるものを **特徴マップ** と呼ぶことがあります。

<img src="figures/conv.gif" width="500">

出典：http://deeplearning.stanford.edu/wiki/images/6/6c/Convolution_schematic.gif

#### 1.2.1 必要な数

#### 1.2.3 良いこと

畳み込み層はMLPのような全結合層だけで構成されたモデルと比較すると、
- 局所結合
- 重み共有

特徴があるためパラメータ数が少なくなるというメリットがあります。

そのため2次元の画像のように、近い画素同士の関係性が強く、離れた画素同士の関係性が低いと考えられるデータに最適です。

##### 局所結合



##### 重み共有

### 1.3 プーリング層 (Pooling Layer)

### 1.4 全結合層 (Dense Layer)